In [1]:
import numpy as np
from numpy.linalg import svd
from sklearn.neighbors import NearestNeighbors


class similaryty_analizer:
    def __init__(self, R: np.array):
        self.R = R
        self.n_users = R.shape[0]
        self.n_items = R.shape[1]

    def _get_svd(self, new_dim: int):
        U, S, V = svd(self.R)

        U = U[:, :new_dim]
        S = np.diag(S[:new_dim])
        V = V[:new_dim, :]

        P = np.matmul(U, S)
        Q = V.T
        return P, Q

    def get_similar_users(self, n_users: int, user_id: int):
        if n_users > self.n_users - 1:
            n_users = self.n_users - 1

        P, Q = self._get_svd(self.n_users)

        nn = NearestNeighbors(n_neighbors=n_users + 1)
        nn = nn.fit(P)

        user = P[user_id, :]
        neighbours = nn.kneighbors(user.reshape(1, -1), return_distance=False)[:, 1:].ravel()
        return neighbours

    def get_similar_items(self, n_items: int, item_id: int):
        if n_items > self.n_items - 1:
            n_items = self.n_items - 1
            
        P, Q = self._get_svd(self.n_items)

        nn = NearestNeighbors(n_neighbors=n_items + 1)
        nn = nn.fit(Q)

        item = Q[item_id, :]
        neighbours = nn.kneighbors(item.reshape(1, -1), return_distance=False)[:, 1:].ravel()
        return neighbours


In [2]:
r1 = [1, 0, 0]
r2 = [1, 1, 0]
r3 = [1, 1, 1]

R = np.array([r1, r2, r3])
R

array([[1, 0, 0],
       [1, 1, 0],
       [1, 1, 1]])

In [3]:
SA = similaryty_analizer(R)
SA.get_similar_users(1, 0)

array([1], dtype=int64)

In [4]:
SA.get_similar_users(2, 0)

array([1, 2], dtype=int64)

In [5]:
SA.get_similar_users(2, 1)

array([0, 2], dtype=int64)